## Importamos e instalamos librerias

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,065 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,323 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launc

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

## Autenticamos con Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Creamos el Spark Context


In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Leemos CSV


In [ ]:
dfCustomers = sqlContext.read.csv('/content/drive/MyDrive/Colab Notebooks/customers.csv', header=True, inferSchema=True)
rddCustomers = dfCustomers.rdd

In [ ]:
dfCategories = sqlContext.read.csv('/content/drive/MyDrive/Colab Notebooks/categories.csv', header=True, inferSchema=True)
rddCategories = dfCategories.rdd

In [ ]:
dfInventoryLogs = sqlContext.read.csv('/content/drive/MyDrive/Colab Notebooks/inventory_logs.csv', header=True, inferSchema=True)
rddInventoryLogs = dfInventoryLogs.rdd

In [ ]:
dfOrderItems = sqlContext.read.csv('/content/drive/MyDrive/Colab Notebooks/order_items.csv', header=True, inferSchema=True)
rddOrderItems = dfOrderItems.rdd

In [ ]:
dfOrders = sqlContext.read.csv('/content/drive/MyDrive/Colab Notebooks/orders.csv', header=True, inferSchema=True)
rddOrders = dfOrders.rdd

In [ ]:
dfProducts = sqlContext.read.csv('/content/drive/MyDrive/Colab Notebooks/products.csv', header=True, inferSchema=True)
rddProducts = dfProducts.rdd

In [ ]:
dfReviews = sqlContext.read.csv('/content/drive/MyDrive/Colab Notebooks/reviews.csv', header=True, inferSchema=True)
rddReviews = dfReviews.rdd

## Pre procesamieno

In [ ]:
#Lo utilizo para ver el dataset claramente
orders_ruta = "/content/drive/MyDrive/Colab Notebooks/orders.csv"
orders = pd.read_csv(orders_ruta)
orders

,Unnamed: 0,order_id,customer_id,order_date,status,payment_method,shipping_address,billing_address,discount_amount,tax_amount,shipping_cost,total_amount,currency,created_at,updated_at,subtotal
0,0,1,447917,2024-07-26T03:04:05.462241,completed,Digital Wallet,"49599 Wesley Burg Richardview, AZ 30649",USNV Morrison FPO AP 90901,0.00,NaN,NaN,25.96,USD,2024-07-26T03:04:05.462241,2024-07-27T03:04:05.462241,0.0
1,1,2,441379,2025-03-22T22:45:14.166543,Completed,Cash on Delivery,"3123 Alan Extension Port Andrea, MA 26926","95271 Russell Spurs Apt. 613 West Marcus, ND 5...",0.00,0.0,43.36,43.36,USD,2025-03-22T22:45:14.166543,NaN,0.0
2,2,3,420585,NaN,Refunded,PayPal,"97369 Brown Cliff Apt. 811 Travisland, ME 61083",9341 VAZQUEZ COMMON SUITE 320 NEW ANTHONYMOUTH...,NaN,0.0,48.59,48.59,USD,2025-02-10T00:33:27.544335,NaN,0.0
3,3,4,456731,2023-11-25T22:45:39.945645,Cancelled,Cash on Delivery,"61010 Adams Ridges Stacymouth, CT 64105","830 Weiss River Suite 861 Lake Nicolemouth, DC...",18.83,0.0,1.39,1.39,USD,NaN,2023-12-02T22:45:39.945645,0.0
4,4,5,468123,2025-02-15T01:58:58.969479,Shipped,Digital Wallet,"495 Fisher Forest Apt. 665 New Scott, AZ 18199","0635 teresa streets apt. 953 robertton, fm 83052",25.38,NaN,NaN,25.25,USD,2025-02-15T01:58:58.969479,2025-02-19T01:58:58.969479,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4699995,4699995,4699996,425063,2025-06-24T18:15:15.531058,Completed,NaN,"4327 Long Lodge Apt. 220 Williamsport, FM 52446","0373 Thomas Forks Gibbsland, MH 78660",0.00,0.0,26.60,26.60,CAD,2025-06-24T18:15:15.531058,2025-06-28T18:15:15.531058,0.0
4699996,4699996,4699997,495953,2024-08-05T05:16:39.939012,Completed,Cash on Delivery,"96377 Jackson Locks Suite 019 South Amber, WY ...","321 valerie lodge west amandabury, az 60836",0.00,NaN,49.28,NaN,EUR,2024-08-05T05:16:39.939012,2024-08-05T05:16:39.939012,NaN
4699997,4699997,4699998,410455,2024-05-23T01:14:29.470059,Cancelled,Debit Card,"384 Thompson Island Apt. 031 Jacksonfort, NH 1...",undefined,78.49,0.0,36.19,36.19,EUR,2024-05-23T01:14:29.470059,2024-05-27T01:14:29.470059,0.0
4699998,4699998,4699999,427581,2024-05-03T22:40:42.433158,Completed,Cash on Delivery,"11562 Todd Knoll Lesliefurt, NY 84930","594 Denise Islands Monicaberg, FL 54231",67.01,0.0,44.16,44.16,EUR,2024-05-03T22:40:42.433158,2024-05-05T22:40:42.433158,0.0


In [ ]:
def safe_lower_strip(x):
    if x is None:
      return x
    else:
      return x.strip().lower()

def safe_upper_strip(x):
    if x is None:
      return x
    else:
      return x.strip().upper()


In [ ]:
rddOrdersAux = rddOrders.map(lambda x: Row(
    id= x[0],
    order_id= x[1],
    customer_id= x[2],
    order_date= x[3],
    status= safe_lower_strip(x[4]),
    payment_method= safe_lower_strip(x[5]),
    shipping_address= safe_upper_strip(x[6]),
    billing_address= safe_upper_strip(x[7]),
    discount_amount= x[8],
    tax_amount= x[9],
    shipping_cost= x[10],
    total_amount= x[11],
    currency= x[12],
    created_at= x[13],
    subtotal= x[14],
))


In [ ]:
#Lo utilizo para ver el dataset claramente
customers_ruta = "/content/drive/MyDrive/Colab Notebooks/customers.csv"
customers = pd.read_csv(customers_ruta)
customers

,Unnamed: 0,customer_id,email,first_name,last_name,phone,date_of_birth,gender,country,city,postal_code,address,registration_date,last_login,is_active,customer_segment,marketing_consent
0,0,1,uthomas@example.net,Kayla,Smith,329-672-0449x89707,1948-05-20,F,Brazil,south michelle,70351.0,NaN,2022-09-22T09:40:47.913063,2025-01-07T12:09:24.323425,True,Regular,True
1,1,2,caldwellandrew@example.com,Carolyn,Gray,(710)769-1350x042,1983-03-19,F,France,Lake Shaneville,65372.0,"247 Allison Overpass Suite 960 North Dillon, A...",2022-11-14T23:20:08.324356,2024-08-25T17:36:03.417619,True,Budget,False
2,2,3,jon32@example.org,Deborah,Martin,+1-694-797-6792,1940-05-23,M,USA,Lisaborough,32855.0,"61358 Mills Spur Lake Tiffany, MS 52073",2022-10-14T15:49:17.728606,2023-04-10T06:15:20.412198,True,budget,True
3,3,4,ksmith@example.org,Rachel,Ellison,NaN,1969-12-21,M,Australia,south michaelborough,11530.0,undefined,2024-03-16T09:18:53.251566,2025-03-20T12:06:38.326651,True,undefined,False
4,4,5,andrewsjessica@example.net,Lisa,Phelps,001-800-821-0991,1940-05-18,M,Japan,Herrerabury,61228.0,"6184 King Trail Lake Trevor, IA 55611",2023-07-21T10:07:22.968497,2025-04-19T11:40:01.441135,True,Regular,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499995,499996,adampeterson@example.com,Kara,Smith,870-523-2459,1942-07-29,NaN,Mexico,North Donald,83228.0,NaN,2022-12-08T21:37:25.538388,undefined,True,Premium,False
499996,499996,499997,walshdevon@example.org,Christopher,Peterson,+1-545-410-3949x0953,1999-12-21,F,Australia,NaN,58730.0,USNV Sherman FPO AA 42084,2023-06-28T04:04:19.964403,undefined,True,Regular,True
499997,499997,499998,freemanlisa@example.com,Joshua,Macdonald,NaN,1985-06-16,undefined,UK,west joshuaborough,2742.0,"96708 Fowler Island Kristinton, OR 21429",2024-04-13T12:31:54.977860,2024-10-16T17:13:41.627636,False,undefined,False
499998,499998,499999,preese@example.com,Jonathan,Brown,+1-700-757-8944,2006-07-21,M,undefined,Robinsonville,NaN,NaN,2022-08-26T11:04:41.091918,2025-03-08T17:06:18.286590,False,REGULAR,True


In [ ]:
rddCustomersAux = rddCustomers.map(lambda x: Row(
    row_number= x[0],
    customer_id= x[1],
    email= x[2],
    first_name= safe_upper_strip(x[3]),
    last_name= safe_upper_strip(x[4]),
    phone= x[5],
    date_of_birth= x[6],
    gender= safe_upper_strip(x[7]),
    country= safe_upper_strip(x[8]),
    city= safe_upper_strip(x[9]),
    postal_code= x[10],
    address= safe_upper_strip(x[11]),
    registration_date= x[12],
    last_login= x[13],
    is_active= x[14],
    customer_segment= safe_lower_strip(x[15]),
    marketing_consent= x[16],
))

In [ ]:
#Lo utilizo para ver el dataset claramente
products_ruta = "/content/drive/MyDrive/Colab Notebooks/products.csv"
products = pd.read_csv(products_ruta)
products

,Unnamed: 0,product_id,product_name,category_id,brand,price,cost,stock_quantity,weight_kg,dimensions,description,is_active,created_at
0,0,1,Centralized attitude-oriented synergy,164,undefined,120.22,49.48,754.0,NaN,194x144x13,Hotel quickly amount care meet. Likely conside...,True,2024-05-12T00:44:32.354038
1,1,2,Reactive asynchronous analyzer,9,Adidas,19.75,10.92,27.0,28.11,188x171x91,Wish white also. Natural career woman across c...,True,2024-06-13T17:16:40.512421
2,2,3,Polarized static flexibility,57,Dr. Martens,454.72,334.06,777.0,40.31,183x113x45,Senior go talk down story forget. Career make ...,True,2025-01-02T18:13:21.175439
3,3,4,Persevering homogeneous Graphic Interface,56,Tiffany & Co.,957.64,331.70,NaN,NaN,93x159x35,Exist little add economy discussion across. Pe...,True,2024-12-15T11:32:56.088835
4,4,5,Compatible non-volatile array,118,IKEA,540.87,311.55,NaN,31.47,97x152x26,Stuff evening north city he sit style poor. Pe...,True,2023-12-05T09:19:11.197502
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999995,999996,Open-architected disintermediate artificial in...,131,Scotts,809.97,438.78,708.0,31.11,181x187x69,Individual common season call country rather. ...,True,NaN
999996,999996,999997,Programmable 6thgeneration strategy,123,Banfield Pet Hospital,631.98,242.30,375.0,38.33,136X23X24,Wife travel reflect professional way fine. Eve...,False,2025-05-11T20:34:52.815172
999997,999997,999998,NaN,20,Wayfair,213.39,103.25,964.0,21.30,189x94x68,Hard less vote sell morning sort foot model. F...,False,2023-09-23T17:08:47.038366
999998,999998,999999,Stand-alone real-time artificial intelligence,78,Staples,790.04,420.99,495.0,30.99,79x56x95,Behind share to sure. Foot reflect race idea f...,True,undefined


In [ ]:
#Sacamos unamed:0 porque no nos aporta info
rddProductsAux = rddProducts.map(lambda x: Row(
    product_id= x[1],
    product_name= safe_upper_strip(x[2]),
    category_id= x[3],
    brand= safe_upper_strip(x[4]),
    price= x[5],
    cost= x[6],
    stock_quantity= x[7],
    weight_kg= x[8],
    dimensions= x[9],
    description= safe_lower_strip(x[10]),
    is_active= x[11],
    created_at= x[12],
))

In [ ]:
#Lo utilizo para ver el dataset claramente
#No le hacemos un pre procesamiento a este rdd ya que solo contiene valores numericos y nuestro pre procesamiento es para strings
#Cualquier filtrado necesario se hace en la consulta que utilice este rdd
order_items_ruta = "/content/drive/MyDrive/Colab Notebooks/order_items.csv"
order_items = pd.read_csv(order_items_ruta)
order_items

,Unnamed: 0,order_item_id,order_id,product_id,quantity,unit_price,line_total,discount_amount
0,0,1,19900399,940377,6.0,569.28,3415.68,0.00
1,1,2,19900399,935931,2.0,1090.20,NaN,0.00
2,2,3,19900399,905111,7.0,NaN,1176.98,154.52
3,3,4,19900399,920065,3.0,464.45,1393.35,NaN
4,4,5,19900400,927289,8.0,283.12,NaN,NaN
...,...,...,...,...,...,...,...,...
285238,285238,285239,19999999,912297,1.0,88.40,88.40,0.00
285239,285239,285240,19999999,945504,10.0,1435.85,14358.50,0.00
285240,285240,285241,20000000,972378,8.0,172.43,1379.44,158.71
285241,285241,285242,20000000,933150,1.0,NaN,811.86,NaN


In [ ]:
reviews_ruta = "/content/drive/MyDrive/Colab Notebooks/reviews.csv"
reviews = pd.read_csv(reviews_ruta)
reviews

,Unnamed: 0,review_id,customer_id,product_id,rating,title,comment,is_verified_purchase,helpful_votes,created_at
0,0,1,473261,933144,5.0,Reveal threat.,Hospital them face left space. Business answer...,True,NaN,2024-06-03T18:27:44.440867
1,1,2,469672,969674,4.0,Voice event soon simple.,Tv owner happen your. Half particularly sort w...,True,61.0,2024-05-28T06:23:34.398247
2,2,3,416180,914709,5.0,Team organization again.,Father door true industry floor.\nOffer off pr...,True,35.0,2023-07-31T18:03:13.899745
3,3,4,466834,906830,4.0,Travel and support financial may.,For near response young newspaper order. Progr...,True,48.0,2024-03-05T10:01:02.678492
4,4,5,490557,956457,5.0,NOTE THESE.,Home reason value sell. Per minute late happen...,False,67.0,2024-05-31T18:08:44.030890
...,...,...,...,...,...,...,...,...,...,...
199995,199995,199996,423897,953150,3.0,Any sense six.,Practice interest coach statement should. Envi...,True,NaN,2023-11-05T14:14:59.243273
199996,199996,199997,462809,954884,4.0,Already some author story well truth sing.,Five national resource why save health researc...,False,NaN,2025-05-20T03:59:35.542485
199997,199997,199998,498895,947766,5.0,NaN,Those wrong fish treatment people customer kin...,True,NaN,2024-11-13T02:20:12.728835
199998,199998,199999,432732,985225,3.0,NaN,Myself meeting when blood month. Still yoursel...,False,42.0,2024-08-29T19:39:51.500896


In [ ]:
rddReviewsAux= rddReviews.map(lambda x: Row(
    review_id= x[1],
    customer_id= x[2],
    product_id= x[3],
    rating= x[4],
    title= safe_lower_strip(x[5]),
    comment= safe_lower_strip(x[6]),
    is_verified_purchase= x[7],
    helpfuel_votes= x[8],
    created_at= x[9],
))

## Consulta 1

In [ ]:
#Cacheo ya que lo voy a utilizar multiples veces
rddOrdersCompleted = rddOrdersAux.filter(lambda x: x.status == "completed").cache()

In [ ]:
import re

#Funcion que nos permite obtener el estado (la libreria fue autorizada por el corrector)

patron_estado = re.compile(r'([A-Z]{2})(?=\s*\d+)')

def get_estado(shipping_address):
    if shipping_address is None:
        return None
    m = patron_estado.search(shipping_address)
    if m:
        return m.group(1)
    return None


In [ ]:
#Chequeo la cantidad de estados
rddOrdersCompleted.filter(lambda x: x.shipping_address != None).map(lambda x: get_estado(x.shipping_address)).distinct().count()

65

In [ ]:
#Como son relativamente pocos hago un collect para ver sí hay algun nombre que debamos quitar (quitamos UNDEFINED ya que no es un estado)
rddOrdersCompleted.filter(lambda x: x.shipping_address != None).map(lambda x: get_estado(x.shipping_address)).distinct().collect()

['MS',
 None,
 'IN',
 'MP',
 'VA',
 'AL',
 'LA',
 'OR',
 'CO',
 'RI',
 'VT',
 'NE',
 'HI',
 'WY',
 'NC',
 'AZ',
 'PR',
 'NY',
 'PA',
 'AR',
 'TN',
 'MT',
 'PW',
 'KY',
 'AE',
 'KS',
 'NM',
 'IL',
 'FL',
 'GU',
 'SD',
 'WI',
 'NV',
 'MO',
 'AK',
 'OH',
 'NJ',
 'CA',
 'ND',
 'SC',
 'UT',
 'IA',
 'MH',
 'TE',
 'NH',
 'WA',
 'ME',
 'CT',
 'TX',
 'AP',
 'DE',
 'MA',
 'MD',
 'DC',
 'IT',
 'VI',
 'WV',
 'AS',
 'MI',
 'OK',
 'FM',
 'ID',
 'GA',
 'MN',
 'AA']

In [ ]:
#Cacheo ya que uso los resultados multiples veces
#Obtengo: (nombre_estado, (total, suma_de_descuentos))

rddCantidadYPromedio = rddOrdersCompleted.filter(lambda x: x.shipping_address != None and x.shipping_address != "UNDEFINED" and x.discount_amount != None).\
map(lambda x: (get_estado(x.shipping_address), (1, x.discount_amount))).\
reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).cache()

In [ ]:
#Top estado por descuento
#Nos quedamos con (nombre_estado, suma_de_descuentos)
rddCantidadYPromedio.map(lambda x: (x[0], x[1][1])).reduce(lambda x,y: x if x[1] > y[1] else y)

('TE', 6427649.680000013)

In [ ]:
#Top por promedio
#Nos quedamos con (nombre_estado, promedio_de_descuento)
rddCantidadYPromedio.map(lambda x: (x[0], x[1][1]/x[1][0])).reduce(lambda x,y: x if x[1] > y[1] else y)

('HI', 12.916488338633958)

## Consulta 2


In [ ]:
#Obtengo: (customer_id, contador)
rddOrdersRefunded = rddOrdersAux.filter(lambda x: x.status == "refunded").map(lambda x: (x.customer_id, 1))

In [ ]:
#Obtengo: (customer_id, (nombre, codigo_postal))
rddCustomersFiltrado= rddCustomersAux.filter(lambda x: x.first_name != None and x.first_name != "UNDEFINED" and x.postal_code != None).map(lambda x: (x.customer_id, (x.first_name, x.postal_code)))

In [ ]:
#Cacheo ya que lo voy a utilizar multiples veces
#Obtengo: (customer_id, (contador, (nombre, codigo_postal)))
rddCustomersOrders = rddOrdersRefunded.join(rddCustomersFiltrado).cache()

#Chequeamos el resultado
rddCustomersOrders.take(5)

[(482210, (1, ('CAROL', 67792))),
 (405440, (1, ('MONICA', 72838))),
 (429720, (1, ('AUTUMN', 92883))),
 (426570, (1, ('MICHAEL', 66428))),
 (496330, (1, ('JEANETTE', 1164)))]

In [ ]:
#5 Codigos Postales mas frecuentes
#[1][1][1] -> codigo postal
Top5PC = rddCustomersOrders.map(lambda x: (x[1][1][1], 1)).reduceByKey(lambda x,y: x+y).takeOrdered(5, lambda x: -x[1])

In [ ]:
#Utilizo Broadcast ya que al tener pocos elementos es mas optimo que hacer un join
bTop5PC = sc.broadcast(Top5PC)

bTop5PC.value

[(15221, 7), (14655, 7), (14025, 7), (72397, 7), (82230, 6)]

In [ ]:
#Personas mas frecuente entre los codigos postales con mayor frecuencia
#[1][1][1] -> codigo postal
#[1][1][0] -> nombre
rddCustomersOrders.filter(lambda x: x[1][1][1] in bTop5PC.value[0]).\
map(lambda x: (x[1][1][0], 1)).reduceByKey(lambda x,y: x+y).reduce(lambda x,y: x if x[1] > y[1] else y)

('JESSICA', 5)

## Consulta 3


In [ ]:
#Obtengo: (customer_id, metodo_de_pago)
rddOrdersFiltrado = rddOrdersAux.filter(lambda x: x.payment_method != None).map(lambda x: (x.customer_id, x.payment_method))

In [ ]:
#Observo la cantidad de tipos que contiene is_active
rddCustomersAux.map(lambda x: type(x.is_active)).distinct().take(5)

[bool]

In [ ]:
#Observo la cantidad de tipos que contiene marketing_consent
rddCustomersAux.map(lambda x: type(x.marketing_consent)).distinct().take(5)

[bool]

In [ ]:
#Al haber solo bools me ahorro el chequeo de Nones para is_active y marketing_consent
#Obtengo: (customer_id, (segmento, es_activo, consentimiento))

rddCustomersFiltrado = rddCustomersAux.filter(lambda x: x.customer_segment != None).\
map(lambda x: (x.customer_id, (x.customer_segment, x.is_active, x.marketing_consent)))

In [ ]:
#Cacheo ya que lo uso multiples veces
#Obtengo: (customer_id, (metodo_de_pago, (segmento, es_activo, consentimiento))
rddCustomersOrders = rddOrdersFiltrado.join(rddCustomersFiltrado).cache()

#Observo el resultado
rddCustomersOrders.take(5)

[(489660, ('bank transfer', ('regular', False, True))),
 (489660, ('bank transfer', ('regular', False, True))),
 (489660, ('cash on delivery', ('regular', False, True))),
 (489660, ('bank transfer', ('regular', False, True))),
 (489660, ('debit card', ('regular', False, True)))]

In [ ]:
#Utilizo take para ver el resultado
#Obtengo: ((paymenth_method, segment_client),(total, total_activos, total_consentimiento)

rddCustomersOrders.map(lambda x: ((x[1][0], x[1][1][0]), (1, 1 if x[1][1][1] is True else 0, 1 if x[1][1][2] is True else 0))) \
.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2])).take(5)

[(('cash on delivery', 'regular'), (389270, 350185, 273204)),
 (('debit card', 'budget'), (126824, 113869, 88403)),
 (('bank transfer', 'premium'), (129544, 116360, 90546)),
 (('digital wallet', 'undefined'), (21521, 19328, 15038)),
 (('digital wallet', 'regular'), (390561, 351015, 274082))]

In [ ]:
#Obtengo: ((paymenth_method, segment_client),(total, total_activos, total_consentimiento)
aux = rddCustomersOrders.map(lambda x: ((x[1][0], x[1][1][0]), (1, 1 if x[1][1][1] is True else 0, 1 if x[1][1][2] is True else 0))) \
.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2])).cache()

In [ ]:
#Como no son muchos datos, entonces, uso el collect para mostrar el resultado
aux.count()

28

In [ ]:
#Obtengo: ((paymenth_method, segment_client), (clientes activos, porcentaje de clientes activos, consentimiento, porcentaje de consentimiento))
aux.map(lambda x: (x[0], (x[1][1], (x[1][1]/x[1][0])*100, x[1][2], (x[1][2]/x[1][0])*100))).collect()

[(('cash on delivery', 'regular'),
  (350185, 89.95941120558994, 273204, 70.18367713926067)),
 (('debit card', 'budget'),
  (113869, 89.78505645619126, 88403, 69.70526083391157)),
 (('bank transfer', 'premium'),
  (116360, 89.82276292225035, 90546, 69.89594269128636)),
 (('digital wallet', 'undefined'),
  (19328, 89.8099530690953, 15038, 69.8759351331258)),
 (('digital wallet', 'regular'),
  (351015, 89.8745650487376, 274082, 70.17648971607508)),
 (('paypal', 'regular'),
  (351110, 89.97214556058661, 274019, 70.21753112804073)),
 (('debit card', 'premium'),
  (117625, 89.79487453528051, 91479, 69.8350293527135)),
 (('digital wallet', 'premium'),
  (117553, 89.80709729172237, 91429, 69.8491157034264)),
 (('undefined', 'premium'),
  (23212, 89.94110353378797, 18172, 70.41227526348419)),
 (('bank transfer', 'undefined'),
  (19415, 89.61458573736442, 15328, 70.7500576967459)),
 (('bank transfer', 'regular'),
  (350102, 89.93416168533756, 273106, 70.15543801873682)),
 (('credit card', 'budg

## Consulta 4

In [ ]:
#Cacheamos ya que lo utilizamos multiples veces
rddProductsStuff = rddProductsAux.filter(lambda x: x.brand != None and x.brand != "UNDEFINED" and x.description != None and ("stuff" in x.description)).cache()

In [ ]:
#Observo la cantidad de marcas
rddProductsStuff.map(lambda x: x.brand).distinct().count()

154

In [ ]:
#Obtengo: (marca, suma de peso_producto * stock_producto)
rddProductsStuff.filter(lambda x: x.weight_kg != None and x.weight_kg > 0 and x.stock_quantity != None and x.stock_quantity > 0).\
map(lambda x: (x.brand, x.stock_quantity * x.weight_kg)).reduceByKey(lambda x,y: x+y).takeOrdered(5, lambda x: -x[1])

[('3M', 1851279.46),
 ('ADIDAS', 1578522.37),
 ('HASBRO', 1506279.13),
 ('WAYFAIR', 1448357.5499999998),
 ('NIKE', 1385209.1700000002)]

## Consulta 5

In [ ]:
#Utilizo cache() ya que voy a usar el resultado multiples veces
rddAux = rddProductsAux.filter(lambda x: x.brand != "UNDEFINED" and x.brand != None and x.stock_quantity != None and x.stock_quantity > 0).cache()

In [ ]:
#Obtengo: (marca, (cant_productos_totales, suma_stock))

#Uso take para ver el resultado
rddAux.map(lambda x: (x.brand, (1, x.stock_quantity))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).take(5)

[('HUGGIES', (4280, 2148722)),
 ('MARVEL', (3743, 1895125)),
 ('MICHAELS', (5188, 2606441)),
 ('CARTIER', (4273, 2141837)),
 ('TOPPS', (5999, 2976653))]

In [ ]:
#Obtengo: (marca, (cant_productos_totales, promedio_stock))
rddMarcas = rddAux.map(lambda x: (x.brand, (1, x.stock_quantity))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0], (x[1][0], x[1][1]/x[1][0])))

#Observo el resultado
rddMarcas.take(5)

[('HUGGIES', (4280, 502.03785046728973)),
 ('MARVEL', (3743, 506.3117819930537)),
 ('MICHAELS', (5188, 502.398033924441)),
 ('CARTIER', (4273, 501.2490053826352)),
 ('TOPPS', (5999, 496.191531921987))]

In [ ]:
#Obtengo: (marca, stock_producto)
rddProductos = rddAux.map(lambda x: (x.brand, x.stock_quantity))

#Observo el resultado
rddProductos.take(5)

[('ADIDAS', 27),
 ('DR. MARTENS', 777),
 ('MOBIL', 959),
 ('HUGGIES', 655),
 ('MARVEL', 274)]

In [ ]:
#Obtengo: (marca, ((cant_productos_totales, promedio_stock), stock_producto))
rddMarcasProductos = rddMarcas.join(rddProductos)

#Observo el resultado
rddMarcasProductos.take(5)

[('MARVEL', ((3743, 506.3117819930537), 274)),
 ('MARVEL', ((3743, 506.3117819930537), 293)),
 ('MARVEL', ((3743, 506.3117819930537), 467)),
 ('MARVEL', ((3743, 506.3117819930537), 433)),
 ('MARVEL', ((3743, 506.3117819930537), 572))]

In [ ]:
#Obtengo: (marca, (cant_productos_totales, cant_productos_superan_20%))

aux = rddMarcasProductos.filter(lambda x: x[1][0][1] + x[1][0][1]*0.2 <= x[1][1]).map(lambda x: (x[0], (x[1][0][0], 1))).\
reduceByKey(lambda x,y: (x[0], x[1] + y[1]))

#Observo el resultado
aux.take(5)

[('MARVEL', (3743, 1484)),
 ('MICHAELS', (5188, 2096)),
 ('CARTIER', (4273, 1699)),
 ('FLUKE', (4351, 1748)),
 ('PEDIGREE', (4322, 1716))]

In [ ]:
#Obtengo: [promedio_pedido]
resultado= aux.map(lambda x: ("total", (x[1][0], x[1][1]))).reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1])).map(lambda x: x[1][1]/x[1][0])

#Es un solo valor asi que uso collect()
#Obs: El resultado de la consulta se muestra aqui
resultado.collect()

[0.39964999074494756]

In [ ]:
#Observo promedio por marca para sacar conclusiones
rddMarcasProductos.filter(lambda x: x[1][0][1] + x[1][0][1]*0.2 <= x[1][1]).map(lambda x: (x[0], (x[1][0][0], 1))).\
reduceByKey(lambda x,y: (x[0], x[1] + y[1])).map(lambda x: (x[0], x[1][1]/x[1][0])).takeOrdered(5, lambda x: -x[1])

[('ROYAL CANIN', 0.40699598203734344),
 ('CARTER’S', 0.4065040650406504),
 ('PLAYMOBIL', 0.406463982470556),
 ('CRAYOLA', 0.4062377402903099),
 ('SKECHERS', 0.4060170394036209)]

## Consulta 6

In [ ]:
rddOrderItemsFiltrado = rddOrderItems.filter(lambda x: x.product_id != None and x.order_id != None and x.quantity != None and x.quantity > 0)

In [ ]:
#Obtengo: (product_id, cantidad)

rddOrdersPorProductID = rddOrderItemsFiltrado.map(lambda x: (x.product_id, x.quantity)).reduceByKey(lambda x,y: x+y)

In [ ]:
#Obtengo top 10 productos mas vendidos
rddTop10Vendidos = rddOrdersPorProductID.takeOrdered(10, lambda x: -x[1])

#Observamos el rdd
rddTop10Vendidos

[(992501, 69),
 (945329, 67),
 (999049, 65),
 (902689, 65),
 (906628, 64),
 (936039, 64),
 (984570, 63),
 (953564, 63),
 (951114, 63),
 (941797, 63)]

In [ ]:
#Utilizamos un broadcast ya que nuestro rdd es pequeño y es mas performante hacerlo de esta manera que con un join
bTop10Vendidos = sc.broadcast(rddTop10Vendidos)

In [ ]:
def filtrar_top(tupla, product_id):
  for elem in tupla:
    if product_id == elem[0]:
      return False
  return True

In [ ]:
#Me devuelve la cantidad de ordenes que no incluyen productos del top 10 vendidos
rddOrderItemsFiltrado.filter(lambda x: filtrar_top(bTop10Vendidos.value, x.product_id)).count()

242369

In [ ]:
#Observo cuantos productos habia en total para sacar conclusiones
rddOrderItemsFiltrado.count()

242457

#Consulta 7

In [ ]:
#Cacheo ya que uso el rdd multiples veces
rddCustomersFiltrado = rddCustomersAux.filter(lambda x: x.country != None and x.country != "UNDEFINED" and x.customer_id != None and x.first_name != None and x.first_name != "UNDEFINED" and \
                                              x.last_name != None and x.last_name != "UNDEFINED").cache()

#Observo el resultado
rddCustomersFiltrado.take(5)

[Row(row_number=0, customer_id=1, email='uthomas@example.net', first_name='KAYLA', last_name='SMITH', phone='329-672-0449x89707', date_of_birth='1948-05-20', gender='F', country='BRAZIL', city='SOUTH MICHELLE', postal_code=70351, address=None, registration_date='2022-09-22T09:40:47.913063', last_login='2025-01-07T12:09:24.323425', is_active=True, customer_segment='regular', marketing_consent=True),
 Row(row_number=1, customer_id=2, email='caldwellandrew@example.com', first_name='CAROLYN', last_name='GRAY', phone='(710)769-1350x042', date_of_birth='1983-03-19', gender='F', country='FRANCE', city='LAKE SHANEVILLE', postal_code=65372, address='247 ALLISON OVERPASS SUITE 960 NORTH DILLON, AZ 64327', registration_date='2022-11-14T23:20:08.324356', last_login='2024-08-25T17:36:03.417619 ', is_active=True, customer_segment='budget', marketing_consent=False),
 Row(row_number=2, customer_id=3, email='jon32@example.org ', first_name='DEBORAH', last_name='MARTIN', phone='+1-694-797-6792', date_of

In [ ]:
#No chequeamos is_active != None porque en la consulta 3 ya nos aseguramos de que solo habia bools
#Obtengo: (pais, (cant_usuarios, cant_usuarios_inactivos))

#Observo el resultado
rddCustomersFiltrado.map(lambda x: (x.country, (1, 1 if not x.is_active else 0))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).take(5)

[('FRANCE', (37808, 3847)),
 ('JAPAN', (37558, 3698)),
 ('GERMANY', (37587, 3711)),
 ('UK', (37143, 3693)),
 ('BRAZIL', (37949, 3766))]

In [ ]:
#Obtengo: (pais, (cant_usuarios, cant_usuarios_inactivos))

top10Paises = rddCustomersFiltrado.map(lambda x: (x.country, (1, 1 if not x.is_active else 0))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0], x[1][1]/x[1][0])).takeOrdered(10, lambda x: -x[1])

#Muestro el resultado de la consulta
top10Paises

[('MEXICO', 0.10370605172094989),
 ('USA', 0.10228271581919399),
 ('FRANCE', 0.10175095217943292),
 ('AUSTRALIA', 0.10065168240457507),
 ('INDIA', 0.09966171573550399),
 ('UK', 0.09942654066715129),
 ('BRAZIL', 0.09923845160610292),
 ('GERMANY', 0.09873094420943411),
 ('JAPAN', 0.09846104691410618),
 ('CANADA', 0.09844696669047807)]

# Consulta 8

In [ ]:
#Obtengo: (customer_id, helpful_votes)
rddReviewsFiltrado = rddReviews.filter(lambda x: x.helpful_votes != None and x.customer_id != None and x.is_verified_purchase and x.is_verified_purchase != None).map(lambda x: (x.customer_id, x.helpful_votes))

In [ ]:
#Esta cacheado 'rddCustomersFiltrado' en la consulta 7
#Obtengo: (customer_id, (pais, nombre))

rddCustomersFiltrado2 = rddCustomersFiltrado.map(lambda x: (x.customer_id, (x.country, x.first_name)))

In [ ]:
#Obtengo: (customer_id, ((pais, nombre), helpful_votes))
rddReviewsCustomers = rddCustomersFiltrado2.join(rddReviewsFiltrado).cache()

#Observo como queda
rddReviewsCustomers.take(5)

[(400028, (('INDIA', 'MORGAN'), 30)),
 (400116, (('JAPAN', 'KARINA'), 47)),
 (400120, (('JAPAN', 'RANDALL'), 43)),
 (400124, (('FRANCE', 'STEVEN'), 59)),
 (400132, (('INDIA', 'GINA'), 15))]

In [ ]:
#Obtengo 'top10Paises' de la consulta 7
bTop10Paises = sc.broadcast(top10Paises)

#Observo el contenido
bTop10Paises.value

[('MEXICO', 0.10370605172094989),
 ('USA', 0.10228271581919399),
 ('FRANCE', 0.10175095217943292),
 ('AUSTRALIA', 0.10065168240457507),
 ('INDIA', 0.09966171573550399),
 ('UK', 0.09942654066715129),
 ('BRAZIL', 0.09923845160610292),
 ('GERMANY', 0.09873094420943411),
 ('JAPAN', 0.09846104691410618),
 ('CANADA', 0.09844696669047807)]

In [ ]:
def filtrar_top_paises(tupla, pais):
  for elem in tupla:
    if pais == elem[0]:
      return True
  return False

In [ ]:
def takeSecond(elem):
    return elem[1]*-1

In [ ]:
def twoMax(ings):
  ings.sort(key=takeSecond)
  return ings[:2]

In [ ]:
#Obtengo: (nombre, cant_helpful_votes)
#x[1][0][0] -> pais
#x[1][0][1] -> nombre
#x[1][1] -> helpful_votes

rddReviewsCustomers.filter(lambda x: filtrar_top_paises(bTop10Paises.value, x[1][0][0])).map(lambda x: (x[1][0][1], x[1][1])).\
reduceByKey(lambda x,y: x+y).map(lambda x: ((x[0],x[1]),('',0))).reduce(lambda x,y: twoMax([x[0],x[1],y[0],y[1]]))

[('MICHAEL', 10305), ('DAVID', 7750)]